### Generates Trip Report for you and your friends

In [1]:
# Install Packages
from IPython.display import clear_output

In [2]:
# Import Packages
from IPython.display import clear_output
import requests
import os
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import time

In [3]:
# Enter trip start date (mm-dd-yyyy)
TripStartDate = "11-26-2024"

# Enter trip end date (mm-dd-yyyy)
TripEndDate = "12-2-2024"

# Formatting and then checking if End date is later than start date
TripStartDate = datetime.strptime(TripStartDate, "%m-%d-%Y")
TripEndDate = datetime.strptime(TripEndDate, "%m-%d-%Y")
if TripEndDate < TripStartDate:
    print("Start date is later than end data, swapping them.")
    temp = TripStartDate
    TripStartDate = TripEndDate
    TripEndDate = temp


In [4]:
# Who was there? (and also ticks)
# Enter your friends who were on the trip here:
FriendsOnTrip = ['Eric','Link','Susan', 'Jack', 'Elizabeth']


# Create array of input csv to be analyzed
FriendsList = []
FriendsCsvFiles = []

FriendsListPath = "./MPcsv/Climbing_Friends.csv"

# Either redownload MP Data or change this to date when you last downloaded from MP
today = datetime.today().strftime("%m-%d-%Y")

with open(FriendsListPath, "r", encoding="utf-8") as file:
    reader = csv.reader(file)
    for row in reader:
        FriendsList.append(row)

#Input csv files to be analyzed
for friend in FriendsList[1:]:
    FriendsCsvFiles.append('./MPcsv/' + friend[0] + "-" + today + ".csv")
    
FriendsThereList = []
FriendInd = []
FoundFriendInd = []
MissingFriends = []
FriendsThereCsvFiles = []
j=0
for Name2LookFor in FriendsOnTrip:
    for i, Name in enumerate(FriendsList):
        if Name2LookFor in Name:  # Check if the keyword is in the string
            FoundFriendInd.append(j)
            FriendInd.append(i)
    j+=1


if len(FoundFriendInd) != len(FriendsOnTrip):
    # Validate indices to ensure they are within range
    valid_indices = set(idx for idx in FoundFriendInd if 0 <= idx < len(FriendsOnTrip))
    # Create a new list excluding the elements at the specified indices
    MissingFriends = [s for i, s in enumerate(FriendsOnTrip) if i not in FoundFriendInd]
    print(MissingFriends, "are missing from your friends list.")
    
# New Friends list with only people there
FriendsThereList = [FriendsList[index] for index in FriendInd]
print(FriendsThereList)
for friend in FriendsThereList:
    FriendsThereCsvFiles.append('./MPcsv/' + friend[0] + "-" + today + ".csv")

# print(FriendsThereCsvFiles)


[['Eric', 'https://www.mountainproject.com/user/200907207/eric-blow'], ['Link', 'https://www.mountainproject.com/user/112051710/link-pat'], ['Susan', 'https://www.mountainproject.com/user/201129089/q-r'], ['Jack', 'https://www.mountainproject.com/user/112192174/jack-reills'], ['Elizabeth', 'https://www.mountainproject.com/user/201346194/elizabeth-yankovsky']]


In [5]:
def check_csv_for_date_range(input_files, output_file, TripStart, TripEnd):
    """
    Python function which goes through N number of csv files and looks at the first column of each file 
    which contains dates, check if those dates are after an entered date range. If they are, then save the 
    rows in which the condition is satisfied
    
    Parameters:
        input_files (list of str): List of file paths to the input CSV files.
        output_file (str): Path to the output CSV file where filtered rows will be saved.
        date_threshold (str): The date threshold in 'YYYY-MM-DD' format. Rows with dates after this will be saved.

    Returns:
        None
    """
    try:
        hasClimbed = []
        routesClimbed = []
        pitchesClimbed = []
        date_series = []
        date_objects = []
        daysOut = []
        routeNames = []
        # Initialize an empty DataFrame to store filtered rows
        filtered_rows = pd.DataFrame()
        i=-1
        for file in input_files:
            # Read the CSV file
            try:
                df = pd.read_csv(file)

                # Ensure the first column contains dates
                if df.empty or len(df.columns) == 0:
                    print(f"Skipping empty or malformed file: {file}")
                    continue
                
                # Convert the first column to datetime
                df[df.columns[0]] = pd.to_datetime(df[df.columns[0]], errors='coerce')

                # Filter rows based on the date condition (between TripStart and TripEnd)
                filtered_df = df[(df[df.columns[0]] >= TripStart) & (df[df.columns[0]] <= TripEnd)]
                i+=1
                # Append filtered rows to the result DataFrame
                if not filtered_df.empty:
                    hasClimbed.append(FriendsThereList[i][0])
                    routesClimbed.append(len(filtered_df))
                    #df ask for pitches and then sum total for each person
                    pitchesClimbed.append(sum(filtered_df['Pitches'].tolist()))
                    # Save Route Names: 
                    routeNames = filtered_df['Route'].tolist()
                    print(FriendsThereList[i][0],'climbed: ', ', '.join(routeNames))
                    # df ask for days and then calc unique days
                    date_series = filtered_df['Date'].tolist()
                    date_objects = [date.strftime("%Y-%m-%d") for date in date_series]
                    unique_days = set(date_objects)    
                    daysOut.append(len(unique_days))
                filtered_rows = pd.concat([filtered_rows, filtered_df], ignore_index=True)
                
            except Exception as e:
                print(f"Error processing file {file}: {e}")

        # Save the filtered rows to the output file
        if not filtered_rows.empty:
            filtered_rows.to_csv('./MPcsv/' + output_file, index=False)
            print(f"\nFiltered rows saved to {output_file}.\n")
        else:
            print("No rows matched the condition.")
        
        print("Trip Summary: ")
        print(
            f"{'Friend':<15}{'Routes':>8}{'Pitches':>11}{'Days Out':>11}"
        )
        print('-' * 50)
        for name, routes, pitches, days in zip(hasClimbed,routesClimbed,pitchesClimbed, daysOut):
            print(
                f"{name:<15}{routes:>5}{pitches:>11}{days:>11}"
            )
        print(f"{'Total':<15}{sum(routesClimbed):>5}{sum(pitchesClimbed):>11}{sum(daysOut):>11}")
    except Exception as e:
        print(f"An error occurred: {e}")

In [6]:
output_file = 'TripReport.csv'
check_csv_for_date_range(FriendsThereCsvFiles, output_file, TripStartDate, TripEndDate)

Eric climbed:  Birdland, Solar Slab, Solar Slab Gully, Dark Shadows, Olive Oil, Kibbles 'n bits, Man's Best Friend
Link climbed:  Birdland, Solar Slab Gully, Solar Slab, Vagabonds, Sweet Pain, Pain Relief, Bon Ez, The CEL, Bear Necessities, Jungle Wall, Tunnel Vision
Susan climbed:  Birdland, Dark Shadows, Olive Oil, Kibbles 'n bits, Man's Best Friend
Jack climbed:  Solar Slab, Dark Shadows, Olive Oil

Filtered rows saved to TripReport.csv.

Trip Summary: 
Friend           Routes    Pitches   Days Out
--------------------------------------------------
Eric               7         33          5
Link              11         33          5
Susan              5         18          4
Jack               3         18          3
Total             26        102         17
